# Распознавание запросов

## Загрузка предобученной модели

In [26]:
!pip install transformers

from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/sbert_large_nlu_ru')
model = AutoModel.from_pretrained('sberbank-ai/sbert_large_nlu_ru')

# В принципе неплохо справляется но не отслеживает варианты слов "задание" | "работа" | "лабораторная"  итд и сильно цепляется за них
# Можно использовать если забить в модели вручную все ети варианты, но... так делать плохо.
# tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
# model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Сама модель с вычислением сходства

In [27]:
class Model:
  queries = [
    'Моя работа принята?',

    # 'Какая разбалловка по дисциплине?',
    'Сколько баллов за семестр по предмету?',

    # 'Какие сроки по лабораторным?',
    # 'Какие сроки сдачи работ?',
    'Какие крайние сроки сдачи работ?',

    'Сколько у меня баллов?',

    'Какое у меня задание?',

    'Когда следующее занятие?',

    'Как зовут преподавателя?',

    'Какая сейчас неделя по счёту?',

    'Когда будет зачётная/контрольная неделя?',
  ]

  # baseline = .5
  baseline = 0.4

  def __init__(self):
    tokens = self.__getTokens__(self.queries)
    embeddings = self.__getEmbeddings__(tokens)
    mask = self.__getMask__(tokens, embeddings)
    self.meanPooled = self.__meanPooling__(embeddings, mask)
    # convert from PyTorch tensor to numpy array
    self.meanPooled = self.meanPooled.detach().numpy()

    # calculate


  def __getTokens__(self, queries):
    # initialize dictionary that will contain tokenized queries
    tokens = {'input_ids': [], 'attention_mask': []}

    for sentence in queries:
        # tokenize sentence and append to dictionary lists
        newTokens = tokenizer.encode_plus(sentence, max_length=128, truncation=True,
                                          padding='max_length', return_tensors='pt')
        tokens['input_ids'].append(newTokens['input_ids'][0])
        tokens['attention_mask'].append(newTokens['attention_mask'][0])

    # reformat list of tensors into single tensor
    tokens['input_ids'] = torch.stack(tokens['input_ids'])
    tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

    return tokens

  def __getEmbeddings__(self, tokens):
    outputs = model(**tokens)
    embeddings = outputs.last_hidden_state

    return embeddings
    
  
  def __getMask__(self, tokens, embeddings):
    attentionMask = tokens['attention_mask']
    mask = attentionMask.unsqueeze(-1).expand(embeddings.size()).float()

    return mask

  def __meanPooling__(self, embeddings, mask):
    maskedEmbeddings = embeddings * mask
    summed = torch.sum(maskedEmbeddings, 1)
    summedMask = torch.clamp(mask.sum(1), min=1e-9)
    meanPooled = summed / summedMask

    return meanPooled
  
  def evaluate(self, userQuery):
    tokens = self.__getTokens__([userQuery])
    embeddings = self.__getEmbeddings__(tokens)
    mask = self.__getMask__(tokens, embeddings)
    meanPooled = self.__meanPooling__(embeddings, mask)
    # convert from PyTorch tensor to numpy array
    result = meanPooled.detach().numpy()

    for n, r in enumerate(result.flatten()):
      print(r, end='\t')
      if (n + 1) % 20 == 0:
        print()
    print()

    similarity = cosine_similarity(
        result,
        self.meanPooled
    )
    similarity = similarity.flatten()
    
    if similarity.max() < self.baseline:
      raise ValueError(f'Запрос не распознан! Максимальная оценка: {similarity.max()}')

    return self.queries[np.argmax(similarity)]

In [28]:
recognitionModel = Model()

## Запросы пользователя

In [29]:
query = '' #@param {type:"string"}

try:
  result = recognitionModel.evaluate(query)
  print(f'Ваш запрос приведён к запросу "{result}"')
except ValueError as e:
  print(e)

-0.31043035	-0.34128112	-0.43965954	-0.03638798	-0.15227666	-0.4711089	0.13404058	-0.2252627	-0.20594805	0.09956046	-0.051587418	0.03202247	0.059778824	-0.27013895	-0.113767065	0.51537216	-0.14106652	0.13455698	0.0366192	0.45287544	
1.3919148	-0.25193146	-0.21002188	0.35410145	-0.53516173	0.6043134	0.035067648	-0.40216872	0.5687205	0.1700823	-0.20100378	-0.73911464	-0.64674085	0.18217942	0.6656282	-0.41296685	-0.034922197	-0.5771822	0.9586755	0.36496362	
0.030926913	0.3433522	-0.44192895	-0.5113566	-0.55755675	-0.19388513	0.33238223	-0.016503194	0.1400208	-0.21287271	-0.015249202	1.193162	-0.62591755	-0.18046069	-0.49617618	0.23737422	-0.55985034	0.3991883	0.01511842	-0.2893096	
0.13711113	-0.19227839	1.0209239	0.4632104	0.044722505	0.119276226	-0.038916595	1.2015034	0.28402698	0.8791945	-1.1927344	-0.030667186	0.9812927	0.10773747	0.08545853	0.84588647	0.14121757	-1.2915854	0.3409825	0.67863095	
0.20456602	0.7695006	0.43764818	0.17576492	-0.44945866	-0.15422368	0.065025255	-0.10306583

In [30]:
query = '\u042F \u043F\u043E\u043B\u0443\u0447\u0438\u043B \u043F\u044F\u0442\u0435\u0440\u043A\u0443?' #@param {type:"string"}

try:
  result = recognitionModel.evaluate(query)
  print(f'Ваш запрос приведён к запросу "{result}"')
except ValueError as e:
  print(e)

1.0391592	-0.41699922	0.10529397	0.5237762	-0.3940909	0.0936987	0.3023846	-0.10078043	0.31037906	-0.15244867	0.3318256	0.47584772	0.396009	0.45699394	-0.9349213	-0.77656573	0.3395171	-0.39693758	-0.19278711	-1.6387469	
0.014113744	0.20434637	-0.5831893	0.5537145	-0.48527393	0.18434437	-0.13487957	-0.54801416	0.9503849	-0.5030186	0.14865951	-0.1486025	0.30510002	-0.31645337	0.8916696	0.6014111	-0.4935396	-0.4146293	0.048639715	0.5187146	
-0.0153162675	0.09144798	-0.5586025	-1.0004488	-0.10353917	-0.047006834	0.38055706	0.108435094	-0.08486453	0.08787105	0.14268748	0.61150557	-0.366159	-0.3235232	0.41003442	-0.8275283	0.11399415	-0.29840115	0.3605884	0.15924029	
-0.22912596	0.10990576	-0.42384282	-0.31968042	0.6434093	-0.062326074	-0.14282407	-1.090273	-0.9097862	0.39664927	0.23708917	0.8453562	0.56371886	-0.21246946	-0.23080109	-0.34200427	-0.4123403	0.6275661	-0.16546856	-0.47920203	
0.052481517	0.22863607	0.31332442	-0.15732442	-0.03298976	0.69565296	-0.3190502	0.27444345	0.8132892	-0

- Моя работа принята?
- Сколько баллов за семестр по предмету?
- Какие крайние сроки сдачи работ?
- Сколько у меня баллов?
- Какое у меня задание?
- Когда следующее занятие?
- Как зовут преподавателя?
- Какая сейчас неделя по счёту?
- Когда будет зачётная/контрольная неделя?


In [31]:
import spacy